### Readme
- The dataset has been uploaded, since registering and requesting access to the data source takes time, and there is no permanent link available. This means there are paths in the notebook that need to be changed according to your Drive structure, these have been marked with a comment.
- The model is built and trained in the .ipynb notebook, to output a .hdf5 model file for the GUI to use. 


- To use the GUI, press 'Browse' and follow the prompts to load the trained model, choose brain modalities to predict, and perform segmentation.
- The folder containing the brain modalities to use/predict when testing the GUI need to contain 4 nii.gz files (for t1, t2, flair, and t1ce) (from the same brain).
- The resulting nii.gz file containing the predicted segmentation is saved in the same parent folder as the folder containing the brain modalities, named as: modalities folder's name + 'prediction.nii.gz'. For example, the folder containing the t1, t2, flair, and t1ce modalities is names 'brain1', then the prediction result is saved as 'brain1prediction.nii.gz' in brain1's parent folder.
- There might be lags/freezes when performing segmentation as the program is taking computation resources. The process should take no longer than 1 minute on decent hardware (37s on i7-10875H CPU).

### Drive operations

In [3]:
"""
Mount Google Drive to access the datasets (datasets must be 
downloaded as there is no permanent link).
"""

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# ! CHANGE this according to your storage structure
%cd "/content/gdrive/MyDrive/Colab Notebooks/Brain"

/content/gdrive/MyDrive/Colab Notebooks/Brain


In [5]:
"""
Extract training data to the runtime's storage instead of Google Drive 
to avoid exceeding operation count and bandwidth quotas.
"""

import zipfile
# ! CHANGE this according to your storage structure
with zipfile.ZipFile('./MICCAI_BraTS2020_TrainingData.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

## **Base code**
Functions to read data, build and train model, and perform segmentation.

### Custom checkpointing
Source: https://github.com/keras-team/keras/issues/12803#issuecomment-614698317

Custom callback, adding the new variable prev_best to ModelCheckpoint that corresponds to the loss of the saved model (inf loss if not specified) to preserve val_loss between Colab timeouts.

In [ ]:
from tensorflow.keras.callbacks import Callback

class SaveModelCheckpoint(Callback):

    def __init__(self, filepath, monitor='val_loss', verbose=0,
                 save_best_only=False, save_weights_only=False,
                 mode='auto', period=1, prev_best=None):
        super(SaveModelCheckpoint, self).__init__()
        self.monitor = monitor
        self.verbose = verbose
        self.filepath = filepath
        self.save_best_only = save_best_only
        self.save_weights_only = save_weights_only
        self.period = period
        self.epochs_since_last_save = 0

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('SaveModelCheckpoint mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'
        
        if(prev_best == None):
            if mode == 'min':
                self.monitor_op = np.less
                self.best = np.Inf
            elif mode == 'max':
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                if 'acc' in self.monitor or self.monitor.startswith('fmeasure'):
                    self.monitor_op = np.greater
                    self.best = -np.Inf
                else:
                    self.monitor_op = np.less
                    self.best = np.Inf
        else:
            if mode == 'min':
                self.monitor_op = np.less
                self.best = prev_best
            elif mode == 'max':
                self.monitor_op = np.greater
                self.best = prev_best
            else:
                if 'acc' in self.monitor or self.monitor.startswith('fmeasure'):
                    self.monitor_op = np.greater
                    self.best = prev_best
                else:
                    self.monitor_op = np.less
                    self.best = prev_best

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0
            filepath = self.filepath.format(epoch=epoch + 1, **logs)
            if self.save_best_only:
                current = logs.get(self.monitor)
                if current is None:
                    warnings.warn('Can save best model only with %s available, '
                                  'skipping.' % (self.monitor), RuntimeWarning)
                else:
                    if self.monitor_op(current, self.best):
                        if self.verbose > 0:
                            print('\nEpoch %05d: %s improved from %0.5f to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch + 1, self.monitor, self.best,
                                     current, filepath))
                        self.best = current
                        if self.save_weights_only:
                            self.model.save_weights(filepath, overwrite=True)
                        else:
                            self.model.save(filepath, overwrite=True)
                    else:
                        if self.verbose > 0:
                            print('\nEpoch %05d: %s did not improve from %0.5f' %
                                  (epoch + 1, self.monitor, self.best))
            else:
                if self.verbose > 0:
                    print('\nEpoch %05d: saving model to %s' % (epoch + 1, filepath))
                if self.save_weights_only:
                    self.model.save_weights(filepath, overwrite=True)
                else:
                    self.model.save(filepath, overwrite=True)

### Read brain modalities

In [ ]:
import os
import numpy as np
import nibabel as nib
from glob import glob

def read_brain(brain_dir, mode='train', x0=42, x1=194, y0=29, y1=221, z0=2, z1=146):

    """
    Reads and crops a brain's modalities (nii.gz format)
    
    Parameters
    ----------
    brain_dir : string
        The path to a folder that contains MRI modalities of a specific brain.
    mode : string
        'train' or 'validation' mode. 
    x0, x1, y0, y1, z0, z1 : int
        The coordinates to crop brain volumes. 
        
    Returns
    -------
    all_modalities : array
        The cropped modalities (+ gt if mode='train').
    brain_affine : array
        The affine matrix of the input brain volume.
    brain_name : str
        The name of the input brain volume.
    """

    # path to the modalities of each brain
    brain_dir = os.path.normpath(brain_dir)
    flair     = glob(os.path.join(brain_dir, '*_flair*.nii.gz'))
    t1        = glob(os.path.join(brain_dir, '*_t1*.nii.gz'))
    t1ce      = glob(os.path.join(brain_dir, '*_t1ce*.nii.gz'))
    t2        = glob(os.path.join(brain_dir, '*_t2*.nii.gz'))
    
    # 'train' mode has an additional ground truth modality
    if mode=='train':
        gt             = glob( os.path.join(brain_dir, '*_seg*.nii.gz'))
        modalities_dir = [flair[0], t1[0], t1ce[0], t2[0], gt[0]]
        
    elif mode=='validation':
        modalities_dir = [flair[0], t1[0], t1ce[0], t2[0]]   
    
    all_modalities = []    # numpy array containing all modalities
    for modality in modalities_dir:      
        nifti_file   = nib.load(modality) # load the NIfTI .nii modality image
        brain_numpy  = np.asarray(nifti_file.dataobj) # create numpy array   
        all_modalities.append(brain_numpy)
        
    # All modalities have the same affine, so take any one of them 
    # Affine is saved for preparing the predicted nii.gz file in the future.   
    # See 'https://nipy.org/nibabel/coordinate_systems.html' for information on
    # coordinate systems and affines in a nifti image    
    brain_affine   = nifti_file.affine
    all_modalities = np.array(all_modalities)
    all_modalities = np.rint(all_modalities).astype(np.int16)
    all_modalities = all_modalities[:, x0:x1, y0:y1, z0:z1]
    # to fit keras channel last model
    all_modalities = np.transpose(all_modalities) 
    brain_name     = os.path.basename(os.path.split(brain_dir)[0]) + '_' + os.path.basename(brain_dir) 

    return all_modalities, brain_affine, brain_name

### Prepare data 

In [ ]:
import os
import tables
import numpy as np
import nibabel as nib
from tqdm import tqdm
from glob import glob
    

def create_table(dataset_dir, table_data_shape, save_dir, crop_coordinates, 
                 data_channels, k_fold=None):
    
    """
    Reads and saves all brain volumes into a single hdf5 table file. 
    
    Parameters
    ----------
    dataset_dir : 
        The path to all brain volumes.
    table_data_shape : tuple
        A tuple which shows the final brain volume shape in the table.
    data_channels : int
        Number of data channels/modalities.
    save_dir : str
        The path to save table.
    crop_coordinates : dict
    k_fold : int
        k-fold cross-validation
        if specified, k .npy files will be saved. Each of these 
        shows the indices of the brain volumes in that fold, 
        which will be used for training the model.
    
    Returns
    -------
    None
    """
    
    all_brains_dir = glob(dataset_dir)  # path to all brain volumes
    all_brains_dir.sort()
    
    hdf5_file    = tables.open_file(os.path.join(save_dir + 'data.hdf5'), mode='w')
    filters      = tables.Filters(complevel=5, complib='blosc')
    data_shape   = tuple([0] + list(table_data_shape) + [data_channels])
    truth_shape  = tuple([0] + list(table_data_shape))
    affine_shape = tuple([0] + [4, 4])
    
    data_storage   = hdf5_file.create_earray(hdf5_file.root, 'data', 
                                             tables.UInt16Atom(), shape=data_shape,
                                             filters=filters, expectedrows=len(all_brains_dir))
    
    truth_storage  = hdf5_file.create_earray(hdf5_file.root, 'truth', 
                                             tables.UInt8Atom(), shape=truth_shape,
                                             filters=filters, expectedrows=len(all_brains_dir))
    
    affine_storage = hdf5_file.create_earray(hdf5_file.root, 'affine', 
                                             tables.Float32Atom(), shape=affine_shape,
                                             filters=filters, expectedrows=len(all_brains_dir))
     
    brain_names = []
    for brain_dir in tqdm(all_brains_dir):
        all_modalities, brain_affine, brain_name = read_brain(brain_dir, mode='train', **crop_coordinates)
        brain    = all_modalities[..., :4]
        gt       = all_modalities[..., -1]  # ground truth label and tumour mask
        
        # in BraTS there is no '3' label
        gt[gt==4]  = 3    
        brain_names.append(brain_name)   
        data_storage.append(brain[np.newaxis,...])
        truth_storage.append(gt[np.newaxis,...])
        affine_storage.append(brain_affine[np.newaxis,...])
        
    hdf5_file.create_array(hdf5_file.root, 'brain_names', obj=brain_names)
    hdf5_file.close()
         
    # for k-fold cross validation
    if k_fold:
        validation_split = (1/k_fold) 
        all_names = [i for i in brain_names]
        
        np.random.seed(100)
        np.random.shuffle(all_names)
              
        val_size = int(validation_split * len(all_names))
        
        for fold in range(k_fold):
            chosen_val = all_names[fold*val_size:(fold+1)*val_size]
        
            chosen_train = [i for i in all_names if i not in chosen_val]
        
            # saving train_idx is enough (val_idx is simply the indices not in train_idx)
            train = chosen_train   
            train_idx = [brain_names.index(i) for i in train]    
            train_idx.sort()
        
            np.save(os.path.join(save_dir, 'fold{}_idx.npy'.format(fold)), train_idx)
    

### Custom data generator

In [ ]:
"""
Realtime multithread data generator
"""

import scipy
import numpy as np
from tensorflow.keras.utils import Sequence, to_categorical
from keras_preprocessing.image.affine_transformations import apply_affine_transform, transform_matrix_offset_center


class CustomDataGenerator(Sequence):
    
    def __init__(self, hdf5_file, brain_idx, batch_size=16, view="axial", 
                 mode='train', horizontal_flip=False, vertical_flip=False, 
                 rotation_range=0, zoom_range=0., shuffle=True):
        """
        Custom data generator based on:
        https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence  
        
        This implementation enables multiprocessing and on-the-fly augmentation 
        which speed up training.
        
        Parameters
        ----------
        hdf5_file : file.File
            hdf5 file that contains all data.
        brain_idx : array
            The brain indices corresponing to a specific fold. All of these 
            brain indices will be use for training and the ones which are 
            not in 'brain_idx' will be used for validation.
        batch_size : int
            The number of input/output arrays that will be generated each 
            time. 
        view : str
            'axial', 'sagittal' or 'coronal'. The generator will extract
            2D slices and perform normalisation with respect to the chosen view.
        mode : str
            Prepare the DataGenerator for 'train' or 'validation' phase. 
        horizontal_flip : bool
            Whether to use horizontal flip for data augmentation.
        vertical_flip : bool
            Whether to use vertical flip for data augmentation.
        rotation_range : float
            Random rotation for data augmentation.
        zoom_range : float
            Random zoom for data augmentation.
        shuffle : bool
            Whether to shuffle data. If mode='validation' 
            shuffling will not be performed.
        """
        
        self.data_storage    = hdf5_file.root.data    # brain data
        self.truth_storage   = hdf5_file.root.truth   # tumour mask
        
        total_brains         = self.data_storage.shape[0]   # total number of brains
        self.brain_idx       = self.get_brain_idx(brain_idx, mode, total_brains)
        self.batch_size      = batch_size
        
        if view == 'axial':
            self.view_axes = (0, 1, 2, 3)            
        elif view == 'sagittal': 
            self.view_axes = (2, 1, 0, 3)
        elif view == 'coronal':
            self.view_axes = (1, 2, 0, 3)            
        else:
            ValueError('unknown input view => {}'.format(view))

        self.mode            = mode
        self.horizontal_flip = horizontal_flip
        self.vertical_flip   = vertical_flip
        self.rotation_range  = rotation_range       
        self.zoom_range      = [1 - zoom_range, 1 + zoom_range]
        self.shuffle         = shuffle
        self.data_shape      = tuple(np.array(self.data_storage.shape[1:])[np.array(self.view_axes)])
        
        print('Using {} out of {} brains'.format(len(self.brain_idx), total_brains), end=' ')
        print('({} out of {} 2D slices)'.format(len(self.brain_idx) * self.data_shape[0], total_brains * self.data_shape[0]))
        print('the generated data shape in "{}" view: {}'.format(view, str(self.data_shape[1:])))
        print('-----'*10)

        self.on_epoch_end()
        
        

    @staticmethod
    def get_brain_idx(brain_idx, mode, total_brains):
        
        """
        Getting the brain indices that will be used by the generator.
        if mode=='train' => the original indices will be used (these npy files 
        were built based on training indices in 'Prepare data' for k-fold)
        if mode=='validation' => the indices which are not in the brain_idx will
        be used.
        """            

        if mode=='validation':
            brain_idx       = np.array([i for i in np.arange(total_brains) if i not in brain_idx])
            print('DataGenerator is preparing for validation mode ...') 
        elif mode=='train':
            brain_idx       = brain_idx
            print('DataGenerator is preparing for training mode ...')
        else:
            raise ValueError('unknown "{}" mode'.format(mode))
            
        return brain_idx


    def __len__(self):
        return int(np.floor( len(self.indices) / self.batch_size))
    
    
    def __getitem__(self, index):

        # Generate indices of the batch
        idx = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X_batch, Y_batch = self.data_load_and_preprocess(idx)

        return X_batch, Y_batch


    def on_epoch_end(self):

        """
        Updates indices after each epoch
        """

        tmp=[]
        for i in self.brain_idx:
            for j in range(self.data_shape[0]):
                tmp.append((i,j))
        self.indices = tmp
            
        if self.mode=='train' and self.shuffle:
            np.random.shuffle(self.indices)
            

    def read_data(self, brain_number, slice_number):
        
        """
        Reads data from table with respect to the 'view'
        """
        
        slice_    = self.data_storage[brain_number].transpose(self.view_axes)[slice_number]
        label_    = self.truth_storage[brain_number].transpose(self.view_axes[:3])[slice_number]
        label_    = np.expand_dims(label_, axis=-1)
        
        return slice_, label_ 


    def data_load_and_preprocess(self, idx):

        """
        Generates data containing batch_size samples
        """

        slice_batch = []
        label_batch = []

        # Generate data
        for i in idx:
            brain_number     = i[0]
            slice_number     = i[1]
            slice_, label_   = self.read_data(brain_number, slice_number)
            slice_           = self.normalise_modalities(slice_)
            slice_and_label  = np.concatenate((slice_, label_) , axis=-1)
            params           = self.get_random_transform()
            slice_and_label  = self.apply_transform(slice_and_label, params)
            slice_           = slice_and_label[...,:4]
            label_           = slice_and_label[..., 4]
            label_           = to_categorical(label_, 4) 
            
            slice_batch.append(slice_)
            label_batch.append(label_)
            
        return np.array(slice_batch), np.array(label_batch)
        
    
    def normalise_slice(self, slice):
        
        """
        Removes 1% of the top and bottom intensities and perform
        normalisation on the input 2D slice.
        """

        b = np.percentile(slice, 99)
        t = np.percentile(slice, 1)
        slice = np.clip(slice, t, b)
        if np.std(slice)==0:
            return slice
        else:
            slice = (slice - np.mean(slice)) / np.std(slice)
            return slice
        
        
    def normalise_modalities(self, Slice): 
        
        """
        Performs normalisation on each modalities of input
        """

        normalised_slices = np.zeros_like(Slice).astype(np.float32)
        for slice_ix in range(4):
            normalised_slices[..., slice_ix] = self.normalise_slice(Slice[..., slice_ix])
    
        return normalised_slices  
    
    
    def get_random_transform(self):

        """
        Randomly generate parameters for affine transformation
        """
    
        if self.rotation_range:
            theta = np.random.uniform(-self.rotation_range,self.rotation_range)    
        else:
            theta = 0            
 
        if self.zoom_range[0] == 1 and self.zoom_range[1] == 1:
            zx, zy = 1, 1
        else:
            zx, zy = np.random.uniform(self.zoom_range[0],self.zoom_range[1], 2)
            
        flip_horizontal = (np.random.random() < 0.5) * self.horizontal_flip    
        flip_vertical   = (np.random.random() < 0.5) * self.vertical_flip
        
        transform_parameters = {'flip_horizontal': flip_horizontal,
                                'flip_vertical':flip_vertical,
                                'theta': theta, 
                                'zx': zx, 
                                'zy': zy}
    
        return transform_parameters    
    

    def flip_axis(self, x, axis):
        
        x = np.asarray(x).swapaxes(axis, 0)
        x = x[::-1, ...]
        x = x.swapaxes(0, axis)
        return x

    
    def apply_transform(self, x, transform_parameters):
        
        """
        Apply affine transformation and axis flip
        """

        x = apply_affine_transform(x, transform_parameters.get('theta', 0),
                           transform_parameters.get('tx', 0),
                           transform_parameters.get('ty', 0),
                           transform_parameters.get('shear', 0),
                           transform_parameters.get('zx', 1),
                           transform_parameters.get('zy', 1),
                           row_axis=0,
                           col_axis=1,
                           channel_axis=2,
                           order=1)
        
        if transform_parameters.get('flip_horizontal', False):
            x = self.flip_axis(x, 1)
        if transform_parameters.get('flip_vertical', False):
            x = self.flip_axis(x, 0)            
        return x    

### Loss
Custom loss function using a combination of generalised dice loss and categorical cross-entropy.

In [ ]:
import tensorflow.keras.backend as bk
from tensorflow.keras.losses import categorical_crossentropy

def generalised_dice(y_true, y_pred):
    
    """
    Generalised Dice Score
    Source: https://arxiv.org/pdf/1707.03237
    """
    
    y_true    = bk.reshape(y_true,shape=(-1,4))
    y_pred    = bk.reshape(y_pred,shape=(-1,4))
    sum_p     = bk.sum(y_pred, -2)
    sum_r     = bk.sum(y_true, -2)
    sum_pr    = bk.sum(y_true * y_pred, -2)
    weights   = bk.pow(bk.square(sum_r) + bk.epsilon(), -1)
    generalised_dice = (2 * bk.sum(weights * sum_pr)) / (bk.sum(weights * (sum_r + sum_p)))
    
    return generalised_dice


def generalised_dice_loss(y_true, y_pred):   
    return 1-generalised_dice(y_true, y_pred)
    
    
def custom_loss(y_true, y_pred):
    
    """
    Sum of generalised dice loss and categorical cross-entropy.
    """
    
    return generalised_dice_loss(y_true, y_pred) + categorical_crossentropy(y_true, y_pred)
    

### Model

In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, PReLU, UpSampling2D, concatenate , Reshape, Dense, Permute, MaxPool2D, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, Activation, add, GaussianNoise, BatchNormalization, multiply
from tensorflow.keras.optimizers import SGD
K.set_image_data_format("channels_last")



def unet_model(input_shape, learning_rate=0.01, start_channel=64, 
               number_of_levels=3, inc_rate=2, output_channels=4, saved_model_dir=None):
    """
    Builds UNet model.
    
    Parameters
    ----------
    input_shape : tuple
        Shape of the input data (height, width, channel).
    learning_rate : float
        Learning rate for the model. 
    start_channel : int
        Number of channels of the first conv layer. 
    number_of_levels : int
        The depth of the U-structure.
    inc_rate : int
        Rate at which the conv channels increase. 
    output_channels : int
        The number of output layer channels. 
    saved_model_dir : str
        If specified, model weights will be loaded from this path. 
    
    Returns
    -------
    model : keras.model
        The created keras model with respect to the input parameters.
    """

        
    input_layer = Input(shape=input_shape, name='input_layer')

    x = GaussianNoise(0.05, name='gaussian_noise')(input_layer)
    x = Conv2D(64, 2, padding='same')(x)
    x = level_block(x, start_channel, number_of_levels, inc_rate)
    x = BatchNormalization(axis = -1)(x)
    x = PReLU(shared_axes=[1, 2])(x)

    x            = Conv2D(output_channels, 1, padding='same')(x)
    output_layer = Activation('softmax')(x)
    
    model        = Model(inputs = input_layer, outputs = output_layer)

    if saved_model_dir:
        model.load_weights(saved_model_dir)
            
    sgd = SGD(lr=learning_rate, momentum=0.9, decay=0)
    model.compile(optimizer=sgd, loss=custom_loss)
    
    return model


def level_block(x, dim, level, inc):
    
    if level > 0:
        m = res_block(x, dim, encoder_path=True)
        x = Conv2D(int(inc*dim), 2, strides=2, padding='same')(m)
        x = level_block(x, int(inc*dim), level-1, inc)

        x = UpSampling2D(size=(2, 2))(x)
        x = Conv2D(dim, 2, padding='same')(x)

        m = concatenate([m,x])
        x = res_block(m, dim, encoder_path=False)
    else:
        x = res_block(x, dim, encoder_path=True)
    return x


def res_block(x, dim, encoder_path=True):

    m = BatchNormalization(axis = -1)(x)
    m = PReLU(shared_axes = [1, 2])(m)
    m = Conv2D(dim, 3, padding='same')(m)

    m = BatchNormalization(axis = -1)(m)
    m = PReLU(shared_axes = [1, 2])(m)
    m = Conv2D(dim, 3, padding='same')(m)
    m = Dropout(0.2)(m)

    if encoder_path:
        x = add([x, m])
    else:
        x = Conv2D(dim, 1, padding='same', use_bias=False)(x)
        x = add([x,m])
    return  x

### Training

In [ ]:
import os
import tables
import numpy as np
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

def train_model(hdf5_dir, brains_idx_dir, view, batch_size=16, val_batch_size=32,
                lr=0.01, epochs=100, hor_flip=False, ver_flip=False, zoom_range=0.0, save_dir='./save/',
                start_chs=64, levels=3, multiprocessing=False, load_model_dir=None, prev_best=None):
    """
    Builds/loads the model, initialises data generators for 
    training and validation, then starts training.
    """
    # preparing generators
    hdf5_file        = tables.open_file(hdf5_dir, mode='r+')
    brain_idx        = np.load(brains_idx_dir)
    train_datagen    = CustomDataGenerator(hdf5_file, brain_idx, batch_size, view, 'train',
                                    hor_flip, ver_flip, zoom_range, shuffle=True)
    val_datagen      = CustomDataGenerator(hdf5_file, brain_idx, val_batch_size, view, 'validation', shuffle=False)
    
    # add callbacks    
    save_dir     = os.path.join(save_dir, '{}_{}'.format(view, os.path.basename(brains_idx_dir)[:5]))
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    logger       = CSVLogger(os.path.join(save_dir, 'log.txt'))
    checkpointer = SaveModelCheckpoint(filepath=os.path.join(save_dir, 'model.hdf5'), verbose=1, 
                                                            save_best_only=True, prev_best=prev_best)
    callbacks    = [logger, checkpointer]        
    
    # building the model
    model_input_shape = train_datagen.data_shape[1:]
    if load_model_dir:
        model = unet_model(model_input_shape, lr, start_chs, levels, saved_model_dir=load_model_dir)
        print("Model weights successfully loaded.")
    else:
        model = unet_model(model_input_shape, lr, start_chs, levels)
    print("The model has been built.")
    model.summary()
    
    # training the model
    model.fit_generator(train_datagen, epochs=epochs, use_multiprocessing=multiprocessing, 
                        callbacks=callbacks, validation_data = val_datagen)
 

### Predict

In [ ]:
import os
import numpy as np
import nibabel as nib
from glob import glob
from tensorflow.keras.models import load_model
    

def normalise_slice(slice):
        
        """
        Removes 1% of the top and bottom intensities and perform
        normalisation on the input 2D slice.
        """

        b = np.percentile(slice, 99)
        t = np.percentile(slice, 1)
        slice = np.clip(slice, t, b)
        if np.std(slice)==0:
            return slice
        else:
            slice = (slice - np.mean(slice)) / np.std(slice)
            return slice


def normalise_volume(input_volume):
    
    """
    Perform a slice-based normalisation on each modalities of input volume.
    """
    normalised_slices = np.zeros_like(input_volume).astype(np.float32)
    for slice_ix in range(4):
        normalised_slices[slice_ix] = input_volume[slice_ix]
        for mode_ix in range(input_volume.shape[1]):
            normalised_slices[slice_ix][mode_ix] = normalise_slice(input_volume[slice_ix][mode_ix])

    return normalised_slices    


def save_predicted_results(prediction, brain_affine, view, output_dir,  
                           z_main=155, z0=2, z1=146, y_main=240, y0=29, y1=221, 
                           x_main=240, x0=42, x1=194):
    
    """
    Save the segmented results into a .nii.gz file.
    To correctly save the segmented brains, it is necessary to set x0, x1,... correctly.
    
    Parameters
    ----------
    prediction : array
        The predictred brain.
    brain_affine : array
        The affine matrix of the predicted brain volume.
    view : str
        'axial', 'sagittal' or 'coronal'. The 'view' is needed to reconstruct output axes.
    output_dir : str
        The path to save .nii.gz file.
    """
    
    prediction = np.argmax(prediction, axis=-1).astype(np.uint16)            
    prediction[prediction==3] = 4
    
    if view=="axial":
        prediction    = np.pad(prediction, ((z0, z_main-z1), (y0, y_main-y1), 
                                            (x0, x_main-x1)), 'constant')
        prediction    = prediction.transpose(2,1,0)
    elif view=="sagital":
        prediction    = np.pad(prediction, ((x0, x_main-x1), (y0, y_main-y1), 
                                            (z0 , z_main-z1)), 'constant')
    elif view=="coronal":
        prediction    = np.pad(prediction, ((y0, y_main-y1), (x0, x_main-x1), 
                                            (z0 , z_main-z1)), 'constant')
        prediction    = prediction.transpose(1,0,2)
    
    prediction_ni    = nib.Nifti1Image(prediction, brain_affine)
    prediction_ni.to_filename(output_dir+ '.nii.gz')
       

## **Execute workflow**
Execute the below cells in order to obtain the trained model and predicted mask. 

**(!)** Compile all base code and the configuration before executing.



### Model training configuration
For the training phase in the workflow.

In [ ]:
cfg = dict()

"""
Model's current best val_loss (for custom checkpointing).
"""
cfg['prev_best']             = 0.69023


# ! CHANGE these paths according to your storage structure
"""
If specified, before training, model weights will be loaded from this path.
If None, train model from scratch.
"""
cfg['load_model_dir']        = './save/axial_fold0/model.hdf5' 


"""
Path to all brain volumes (use '/*' since each brain's modalities are stored 
in a folder within the 'MICCAI_BraTS2020_TrainingData' folder)
"""
cfg['data_dir']              = '/content/MICCAI_BraTS2020_TrainingData/*'


# the 'data' and 'save' folders will be created
"""
Path to save table file + k-fold files
"""
cfg['save_data_dir']         = './data/'


"""
Path to save models + log files + tensorboards
"""
cfg['save_dir']              = './save/'


"""
Default path of saved table.
"""
cfg['hdf5_dir']              = './data/data.hdf5'


"""
Path to brain indices of a specific fold (if using k-fold)
"""
cfg['brains_idx_dir']        = './data/fold0_idx.npy'


"""
BraTS datasets contain 4 channels: (FLAIR, T1, T1ce, T2)
"""
cfg['data_channels']         = 4


"""
k-fold cross-validation
"""
cfg['k_fold']                = 5



"""
Coordinates to crop brain volumes.  
Final three shapes must be divisible by the network downscale rate.
"""
cfg['crop_coord']            = {'x0':42, 'x1':194,
                                'y0':29, 'y1':221,
                                'z0':2,  'z1':146}


"""
Final data shapes of saved table file.
"""
cfg['table_data_shape']      = (cfg["crop_coord"]['z1']-cfg["crop_coord"]['z0'],
                                cfg["crop_coord"]['y1']-cfg["crop_coord"]['y0'], 
                                cfg["crop_coord"]['x1']-cfg["crop_coord"]['x0'])


"""
'axial', 'sagittal' or 'coronal'. 
All 2D slices and the model will be prepared with respect to 'view'.
"""
cfg['view']                  = 'axial'


"""
Training and validation batch size
"""
cfg['batch_size']            = 32
cfg['val_batch_size']        = 48


"""
Augmentation parameters.
"""
cfg['hor_flip']              = True
cfg['ver_flip']              = True
cfg['rotation_range']        = 0
cfg['zoom_range']            = 0.


"""
Learning rate and the number of epochs for training the model.
"""
cfg['epochs']                = 100
cfg['lr']                    = 0.008


"""
If True, use multithreading.
"""
cfg['multiprocessing']       = False 


"""
Maximum number of processes when multithreading. 
If unspecified, workers default to 1. If 0, execute the generator 
on the main thread. 
"""
cfg['workers']               = 10


"""
Depth of the U-Net architecture. 
"""
cfg['levels']                = 3


"""
Number of channels of the first conv layer.
"""
cfg['start_chs']             = 64

### Read and prepare data
**(!)** Delete the 2 .csv files in the extracted folder so that the function does not read them.

In [ ]:
"""
Reads and saves all brain volumes into a single table file.
"""

create_table(cfg['data_dir'], cfg['table_data_shape'], cfg['save_data_dir'], 
             cfg['crop_coord'], cfg['data_channels'], cfg['k_fold'])

100%|██████████| 369/369 [05:59<00:00,  1.03it/s]


### Train the model

In [ ]:
train_model(cfg['hdf5_dir'], cfg['brains_idx_dir'], cfg['view'], cfg['batch_size'], 
            cfg['val_batch_size'], cfg['lr'], cfg['epochs'], cfg['hor_flip'], 
            cfg['ver_flip'], cfg['zoom_range'], cfg['save_dir'], cfg['start_chs'], 
            cfg['levels'], cfg['multiprocessing'], cfg['load_model_dir'], 
            cfg['prev_best'])

DataGenerator is preparing for training mode ...
Using 296 out of 369 brains (42624 out of 53136 2D slices)
the generated data shape in "axial" view: (192, 152, 4)
--------------------------------------------------
DataGenerator is preparing for validation mode ...
Using 73 out of 369 brains (10512 out of 53136 2D slices)
the generated data shape in "axial" view: (192, 152, 4)
--------------------------------------------------


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Model weights successfully loaded.
The model has been built.
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 192, 152, 4) 0                                            
__________________________________________________________________________________________________
gaussian_noise (GaussianNoise)  (None, 192, 152, 4)  0           input_layer[0][0]                
__________________________________________________________________________________________________
conv2d_75 (Conv2D)              (None, 192, 152, 64) 1088        gaussian_noise[0][0]             
__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 192, 152, 64) 256         conv2d_75[0][0]                  
_______________________________

KeyboardInterrupt: ignored

### Predict

In [ ]:
"""
Extract testing data to the runtime's storage instead of Google Drive 
to avoid exceeding operation count and bandwidth quotas.
"""

import zipfile
# ! CHANGE this according to your storage structure
with zipfile.ZipFile('/content/gdrive/MyDrive/Colab Notebooks/42028/Assignment 3/MICCAI_BraTS2020_ValidationData.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
"""
Obtain the predicted tumour masks for the test dataset.
"""
# ! CHANGE these paths according to your storage structure
val_data_dir       = '/content/MICCAI_BraTS2020_ValidationData/*'
saved_model_dir    = '/content/gdrive/MyDrive/Colab Notebooks/42028/Assignment 3/save/axial_fold0/model.hdf5'

# 'predict' folder will be created
save_pred_dir      = './predict/'
view               = 'axial'
batch_size         = 32


if not os.path.isdir(save_pred_dir):
    os.mkdir(save_pred_dir)
    
all_brains_dir = glob(val_data_dir)
all_brains_dir.sort()

if view == 'axial':
    view_axes = (0, 1, 2, 3)            
elif view == 'sagittal': 
    view_axes = (2, 1, 0, 3)
elif view == 'coronal':
    view_axes = (1, 2, 0, 3)            
else:
    ValueError('unknown input view => {}'.format(view))

model        = load_model(saved_model_dir, compile=False)
for brain_dir in all_brains_dir:    
    if os.path.isdir(brain_dir):
        print("Volume ID: ", os.path.basename(brain_dir))
        all_modalities, brain_affine, _ = read_brain(brain_dir, mode='validation')
        all_modalities                  = all_modalities.transpose(view_axes)
        all_modalities                  = normalise_volume(all_modalities)
        prediction                      = model.predict(all_modalities, batch_size=batch_size, verbose=1)
        output_dir                      = os.path.join(save_pred_dir, os.path.basename(brain_dir))
        save_predicted_results(prediction, brain_affine, view, output_dir)

Volume ID:  BraTS20_Validation_001
5/5 [==============================] - 2s 393ms/step
Volume ID:  BraTS20_Validation_002
5/5 [==============================] - 1s 227ms/step
Volume ID:  BraTS20_Validation_003
5/5 [==============================] - 1s 227ms/step
Volume ID:  BraTS20_Validation_004
5/5 [==============================] - 1s 225ms/step
Volume ID:  BraTS20_Validation_005
5/5 [==============================] - 1s 228ms/step
Volume ID:  BraTS20_Validation_006
5/5 [==============================] - 1s 226ms/step
Volume ID:  BraTS20_Validation_007
5/5 [==============================] - 1s 227ms/step
Volume ID:  BraTS20_Validation_008
5/5 [==============================] - 1s 226ms/step
Volume ID:  BraTS20_Validation_009
5/5 [==============================] - 1s 226ms/step
Volume ID:  BraTS20_Validation_010
5/5 [==============================] - 1s 228ms/step
Volume ID:  BraTS20_Validation_011
5/5 [==============================] - 1s 227ms/step
Volume ID:  BraTS20_Validation_0

### GUI code

In [ ]:
# Import Required Libraries

from tkinter import *
from tkinter import messagebox
from tkinter import filedialog
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import os
import nibabel as nib
from glob import glob
import cv2
from tensorflow.keras.models import load_model
# import cv2 <-- To use openCV function/methods

# Create a Window.
MyWindow = Tk() # Create a window
MyWindow.title("Brain Tumour Segmentation") # Change the Title of the GUI
MyWindow.geometry('1024x768') # Set the size of the Windows


header_label = Label(MyWindow)
header_label.place(relx=.5, rely=0.1, anchor='center')
word = 'Brain Tumour Segmentation'
header_label.configure(text = word, font= ("Arial", 42))



def read_brain(brain_dir, mode='train', x0=42, x1=194, y0=29, y1=221, z0=2, z1=146):

    """
    Reads and crops a brain's modalities (nii.gz format)
    
    Parameters
    ----------
    brain_dir : string
        The path to a folder that contains MRI modalities of a specific brain
    mode : string
        'train' or 'validation' mode. 
    x0, x1, y0, y1, z0, z1 : int
        The coordinates to crop brain volumes. 
        
    Returns
    -------
    all_modalities : array
        The cropped modalities (+ gt if mode='train')
    brain_affine : array
        The affine matrix of the input brain volume
    brain_name : str
        The name of the input brain volume
    """

    # path to the modalities of each brain
    brain_dir = os.path.normpath(brain_dir)
    flair     = glob(os.path.join(brain_dir, '*_flair*.nii.gz'))
    t1        = glob(os.path.join(brain_dir, '*_t1*.nii.gz'))
    t1ce      = glob(os.path.join(brain_dir, '*_t1ce*.nii.gz'))
    t2        = glob(os.path.join(brain_dir, '*_t2*.nii.gz'))
    
    # 'train' mode has an additional ground truth modality
    if mode=='train':
        gt             = glob( os.path.join(brain_dir, '*_seg*.nii.gz'))
        modalities_dir = [flair[0], t1[0], t1ce[0], t2[0], gt[0]]
        
    elif mode=='validation':
        modalities_dir = [flair[0], t1[0], t1ce[0], t2[0]]   
    
    all_modalities = []    # numpy array containing all modalities
    for modality in modalities_dir:      
        nifti_file   = nib.load(modality) # load the NIfTI .nii modality image
        brain_numpy  = np.asarray(nifti_file.dataobj) # create numpy array   
        all_modalities.append(brain_numpy)
        
    # all modalities have the same affine, so we take one of them (the last one in this case),
    # affine is just saved for preparing the predicted nii.gz file in the future.       
    brain_affine   = nifti_file.affine
    all_modalities = np.array(all_modalities)
    all_modalities = np.rint(all_modalities).astype(np.int16)
    all_modalities = all_modalities[:, x0:x1, y0:y1, z0:z1]
    # to fit keras channel last model
    all_modalities = np.transpose(all_modalities) 
    # tumor grade + name
    brain_name     = os.path.basename(os.path.split(brain_dir)[0]) + '_' + os.path.basename(brain_dir) 

    return all_modalities, brain_affine, brain_name


def normalize_slice(slice):
        
        """
        Removes 1% of the top and bottom intensities and perform
        normalization on the input 2D slice.
        """
        b = np.percentile(slice, 99)
        t = np.percentile(slice, 1)
        slice = np.clip(slice, t, b)
        if np.std(slice)==0:
            return slice
        else:
            slice = (slice - np.mean(slice)) / np.std(slice)
            return slice


def normalize_volume(input_volume):
    
    """
    Perform a slice-based normalization on each modalities of input volume.
    """
    normalized_slices = np.zeros_like(input_volume).astype(np.float32)
    for slice_ix in range(4):
        normalized_slices[slice_ix] = input_volume[slice_ix]
        for mode_ix in range(input_volume.shape[1]):
            normalized_slices[slice_ix][mode_ix] = normalize_slice(input_volume[slice_ix][mode_ix])

    return normalized_slices    


def save_predicted_results(prediction, brain_affine, view, output_dir,  
                           z_main=155, z0=2, z1=146, y_main=240, y0=29, y1=221, 
                           x_main=240, x0=42, x1=194):
    
    """
    Save the segmented results into a .nii.gz file.
    To correctly save the segmented brains, it is necessery to set x0, x1, ... correctly.
    
    Parameters
    ----------
    prediction : array
        The predictred brain.
    brain_affine : array
        The affine matrix of the predicted brain volume
    view : str
        'axial', 'sagittal' or 'coronal'. The 'view' is needed to reconstruct output axes.
    output_dir : str
        The path to save .nii.gz file.
    """
    
    prediction = np.argmax(prediction, axis=-1).astype(np.uint16)            
    prediction[prediction==3] = 4
    
    if view == "axial":
        prediction    = np.pad(prediction, ((z0, z_main-z1), (y0, y_main-y1), (x0, x_main-x1)), 'constant')
        prediction    = prediction.transpose(2,1,0)
    elif view == "sagital":
        prediction    = np.pad(prediction, ((x0, x_main-x1), (y0, y_main-y1), (z0 , z_main-z1)), 'constant')
    elif view == "coronal":
        prediction    = np.pad(prediction, ((y0, y_main-y1), (x0, x_main-x1), (z0 , z_main-z1)), 'constant')
        prediction    = prediction.transpose(1,0,2)
    #
    prediction_ni    = nib.Nifti1Image(prediction, brain_affine)
    prediction_ni.to_filename(output_dir + 'prediction.nii.gz')
       

view               = 'axial'

if view == 'axial':
    view_axes = (0, 1, 2, 3)            
elif view == 'sagittal': 
    view_axes = (2, 1, 0, 3)
elif view == 'coronal':
    view_axes = (1, 2, 0, 3)            
else:
    ValueError('unknown input view => {}'.format(view))

        
def predict_single(saved_model, folder):
    model        = load_model(saved_model, compile=False)    
    if os.path.isdir(folder):
        print("Volume ID: ", os.path.basename(folder))
        all_modalities, brain_affine, _ = read_brain(folder, mode='validation')
        all_modalities                  = all_modalities.transpose(view_axes)
        all_modalities                  = normalize_volume(all_modalities)
        prediction                      = model.predict(all_modalities, verbose=1)
        save_predicted_results(prediction, brain_affine, view, folder)
        open_image(folder + 'prediction.nii.gz')


# Methods for processing the brain modalities using trained model

def read_img(img_path):
    return sitk.GetArrayFromImage(sitk.ReadImage(img_path))


def open_folder(folder):
    messagebox.showinfo("Chosen Folder", folder)
    t1 = glob(os.path.join(folder,'*t1.nii.gz'))
    t2 = glob(os.path.join(folder,'*t2.nii.gz'))
    flair = glob(os.path.join(folder,'*flair.nii.gz'))
    t1ce = glob(os.path.join(folder,'*t1ce.nii.gz'))
    for i in t1:
        img = (read_img(i)[100]).astype(np.uint8)
        cv2.imshow('brain', img)

# Open Image Function using OpenCV
def open_image(image):
    messagebox.showinfo("Image to Show", image)
    # Open the image using OpenCV
    img = nib.load(image)
    data = img.get_fdata()
    cv2.imshow('segmentation', data[:, :, data.shape[2] // 2].T)


# Event Methods attached to the buttons
def BttnOpen_Clicked():
    messagebox.showinfo("Info", "Select the pretrained model.")
    saved_model = filedialog.askopenfilename(filetypes = (("hdf5 files","*.hdf5"),("h5 files","*.h5"),("pb files","*.pb")))
    messagebox.showinfo("Info", "Pretrained model selected.")
    
    messagebox.showinfo("Info", "Choose folder containing the brain modalities (4 nii.gz files).")
    folder = filedialog.askdirectory()
    open_folder(folder) 
    messagebox.showinfo("Info", "A modality slice has been shown in the newly opened window (brain).")
    
    messagebox.showinfo("Info", "Click OK to start segmentation. A popup will show when the segmentation process has been completed.")
    predict_single(saved_model, folder)
    
    messagebox.showinfo("Info", "Segmentation finished. The predicted tumour mask is displayed (segmentation) and the " +
                        "resulting nii.gz file has been saved in the same parent folder as the folder containing the brain modalities.")


def BttnProcess_Clicked():
    file = filedialog.askopenfilename(filetypes = (("NifTI files","*.nii.gz"),("all files","*.*")))
    open_image(file)
    
    # messagebox.showinfo("Info", "Process Button Clicked")
    # predict_single(folder)
    
    
    # Testing
    # messagebox.showwarning("Invalid Input","Image is having an invalid format") # Showing Warning not very Critcal 
    # messagebox.showerror("Invalid Input","Image is having an invalid format") # Showing Error, very Critcal 
    # classifcationResult = "CAT"
    # messagebox.showinfo("Classfication Result", classifcationResult)
    # result = "DOG"
    # resultText = "Classification Result:" + result  # Concatenate the result class to the Label on the Window
    # ClassficationResultLabel.configure(text = resultText)  # Update the Label text on the Window
    
    

# Add the Components created previsously to the window

InputLabel = Label(text="Input Image")
InputLabel.place(x=300, y=200)
SegLabel = Label(text="Segmented Image")
SegLabel.place(x=600, y=200)
BrowseBttn = Button(text="Browse", command=BttnOpen_Clicked)
BrowseBttn.place(x=0, y=180)
ProcessBttn = Button(text="Tumour Area %", command=BttnProcess_Clicked)
ProcessBttn.place(x=300, y=500)
ProcessBttn = Button(text="Tumour Grade", command=BttnProcess_Clicked)
ProcessBttn.place(x=600, y=500)

# Calling the mainloop()
MyWindow.mainloop()
